# Intro (소개)

This notebook is written by English and Korean.

Let's analyze and visualize the titanic survivors's data to get variety insight of titanic.

There are some large graphs. Watch original size if you can't see well.

There is the prediction model too but the main content is analyzing.

Welcome feedback, even typo. Plz leave a comment.

---

이 노트북은 영어와 한국어로 쓰여졌습니다.

타이타닉 사건의 여러 관점을 얻기 위해 데이터를 분석하고 시각화 해봅시다.

큰 그래프들이 몇개 있습니다. 화면이 작아 잘 안보이면 원본 사이즈로 봐보세요.

예측 모델도 있긴하지만 주요 컨텐츠는 데이터 분석입니다.

피드백은 환영합니다. 댓글 남겨주세요.

# Exploring the data (데이터 살펴보기)

In [ ]:
# load moudles
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# load csv file load
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

## First of all, let's check the data, get some ideas, and find what we have to do

## 우선, 데이터를 확인하고 아이디어를 얻은 다음 무엇을 해야될지 알아봅시다.

In [ ]:
df_train.shape

There are 891 passengers and 12 features.

---

891명의 승객과 12개의 특징이 있습니다.

In [ ]:
df_train.head(5)

There is no specially noticeable data

check [titanic/data](https://www.kaggle.com/c/titanic/data) if you want to know what each column meaning

---

딱히 눈에 띄는 데이터는 없습니다.

각 열이 무엇을 의미하는지 알고 싶다면 [titanic/data](https://www.kaggle.com/c/titanic/data)를 참고하세요.

In [ ]:
df_train.describe()

There are huge differences between 75\% and max value in `SibSp` (Number of siblings and spouse aboard the titanic), `Parch` (Number of parcent and children aboard the titanic), and `Fare`.

Most people brought only 0 or 1 family member. Judging from those facts, titanic was not a family-friendly tour.

And the passenger's fare's mean value is 32\$. Considering the cost at that time, it was not cheap. I guess it wouldn't be a light tour that go alone.

I think there were more unofficially relationships like friends or lovers than families.

---

 `SibSp` (타이타닉에 탑승한 형제자매와 배우자의 수), `Parch` (타이타닉에 탑승한 부모님과 자식의 수), and `Fare` (요금) 의 75%와 최고치 사이에 큰 차이가 보입니다.
 
 대부분의 사람들이 가족들을 0명 혹은 1명까지만 데려온 것입니다. 이점을 미루어 볼 때 타이타닉은 가족들이 갈만한 여행은 아니였습니다.
 
 그리고 승객들이 낸 요금들을 보면 평균 32\\$이고 그 당시의 물가를 생각하면 싼가격이 아닙니다. 제 생각에는 혼자 가볍게 갈 만한 여행은 아니었을겁니다.
 
 제 생각에는 친구나 연인같은 비공식적인 관계가 가족들보다 많았던 것 같습니다.

---

reference (참고) : [cost 100 years ago](https://www.countryliving.com/life/g33398396/what-things-cost-100-years-ago/?slide=3)

In [ ]:
df_train.isna().sum()

Last, there are missing values in `Cabin`, `Age`, and `Embarked`. Especially `Cabin`. we have to consider how to handle or fill that empty values but need more insight for now.

---

마지막으로, `Cabin`, `Age`, and `Embarked` 에 없는 값들이 보입니다. 특히 `Cabin`이 말이죠. 이 빈값들을 어떻게 조정 혹은 채워나갈지 고민해봐야겠네요. 하지만 지금은 더 많은 관점으로 바라볼 필요가 있습니다.

# Examine each column (각 열 살펴보기)

There is nothing to handle with `PassengerId`, `Survived`, `Pclass`, `Name`, `SibSp`, and `Parch`. So let them leave as they are.

---

`PassengerId`, `Survived`, `Pclass`, `Name`, `SibSp`, and `Parch` 는 딱히 건들게 없습니다. 그래서 있는 그대로 놔두겠습니다.

## Survived (생존여부)

In [ ]:
print(df_train['Survived'].value_counts())
sns.countplot(x=df_train['Survived'])

First, it's a graph about number of survivors and non-survivors. Sadly, more than half didn't make it. Let's keep going.

---

먼저, 생존자와 아닌 사람들의 수에 대한 그래프입니다. 슬프게도, 반 이상의 사람들이 죽었습니다. 계속 살펴봅시다.

## Sex (성별)

Actually, `Sex` isn't much to handle. But categorialization will be helpful to deal with it later.

---

사실 `Sex`는 딱히 건들 것은 없습니다. 하지만 카테고리화는 나중에 값을 처리하는데 도움이 될 것 같네요.

In [ ]:
 df_train['Sex'] = df_train['Sex'].astype('category')
print(df_train['Sex'])

## Age (나이)

There are lots of ages from 0 to 80. But the result has only two options, live or die. And at that disaster, people didn't calculate 1 or 2 years to select who have to live. I think it's not a good idea that use age feature as it is. So, I grouped `Age` field by 3 age groups. `kid` (younger than 16), `adult` (between 16, 55), and `elder` (older than 55). I think standard of adult was younger than now so classify over 15 as `adult`. And other one is grouped as divide into ten.

Under below is grahs that show the survival rate that each grouped age group.

Obviously, kids were much more survived than adults while about 45% twenties were dead.

---

나이는 0세부터 80세까지 수많은 나이들이 있습니다. 하지만 결과는 오직 두가지, 살거나 죽거나입니다. 그리고 그런 재앙에서 사람들이 누구를 살릴지 정하기 위해 1, 2년을 계산하지는 않았을겁니다. 제 생각에는 지금 나이를 있는 그대로 사용하는건 별로 좋은 생각은 아닌 것 같습니다. 그래서, `Age` 필드를 3개의 연령대 나눠봤습니다. `kid` (16 아래), `adult` (16과 55 사이), and `elder` (55보다 높음). 그때는 지금보다 어른의 기준이 더 어릴 것이라 생각해서 15세 위로는 어른으로 분류했습니다. 그리고 다른 하나는 10으로 나눠서 그룹화했습니다.

아래는 그룹화한 나이대의 생존률을 그래프로 나타낸 것입니다.

확실히, 아이들이 어른들보다 많이 생존했고 반대로 20대들 중 약 45%는 사망했네요.

In [ ]:
def groupAgeByTen (row) :
    if (np.isnan(row.Age)) :
        return 'unidentified'
    return int(row.Age / 10)
df_train['AgeGroupByTen'] = df_train.apply(groupAgeByTen, axis=1)
df_train['AgeGroupByTen'] = df_train['AgeGroupByTen'].astype('category')

def groupAgeByTitle (row) :
    if (row.Age < 16) :
        return 1
    if (row.Age >= 16 and row.Age < 55) :
        return 2
    if (row.Age > 55) :
        return 3
    return 0
df_train['AgeGroupByTitle'] = df_train.apply(groupAgeByTitle, axis=1)
df_train['AgeGroupByTitle'] = df_train['AgeGroupByTitle'] \
    .astype('category') \
    .cat \
    .rename_categories(['unidentified', 'kid', 'adult', 'elder'])

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.countplot(x='AgeGroupByTen', hue='Survived', data=df_train, ax=axes[0])
sns.countplot(x='AgeGroupByTitle', hue='Survived', data=df_train, ax=axes[1])

## Ticket (티켓, 표)

The `Ticket` has 681 unique values. No significant pattern has found. I'll remove it.

---

`Ticket`은 고유한 681개의 값을 가지고 있습니다. 딱히 의미있는 패턴은 찾을 수 없었습니다. 삭제하도록 하겠습니다.

In [ ]:
print('Number of Unique values in `Ticket` : ', len(df_train.groupby('Ticket').groups.keys()))
print('`Ticket` 필드에 있는 고유한 값의 수 : ', len(df_train.groupby('Ticket').groups.keys()))
df_train = df_train.drop('Ticket', axis=1)

## Fare (요금)

In fact, there is nothing to handle `Fare` too. But I found some interesting.

---

사실 `Fare`도 딱히 건들만한건 없습니다. 하지만 몇 가지 흥미로운 점을 발견했습니다.

In [ ]:
df_train.sort_values('Fare', ascending=False).head(15)

The table above shows us the top 15 passengers that paid the highest fare. 

If you see `Cabin` and `Fare`, you can guess that passengers who paid the same price stayed in the same cabin. From that guess, it might be possible to fill `NaN` value in `Cabin`. But, I'll let them as it is for now.

And 3 passengers from the top have a huge fare differences with the 4th passenger. It seems to be the biggest reason for the defference between 75% and max value.

---

위 표는 가장 많은 요금을 낸 상위 15명의 승객들을 보여줍니다.

요금과 객실을 보시면 같은 요금을 낸 승객이 같은 객실에 묵었다는 것을 추측해볼 수 있습니다. 그 추측을 기반으로 `Cabin`의 `NaN` 값을 채워넣을 수도 있겠네요. 하지만 일단은 그대로 놔두겠습니다.

그리고 위에서 3명의 승객들은 4 번째 승객과 요금이 거의 2배 차이가 나네요. 이 승객들이 75% 와 max 값에 차이를 준 가장 큰 원인 같습니다.

In [ ]:
df_train['Fare'] = (df_train['Fare'] / 10).astype(int)

In [ ]:
plt.figure(figsize=(30, 6))
sns.countplot(x='Fare', hue='Survived', data=df_train)
plt.legend(loc='upper right')
plt.show()

plt.figure(figsize=(30, 6))
df_train.groupby('Fare').mean()['Survived'].plot()
plt.show()

Graph above shows the passengers fare in 10 $ increments. The survival rate is getting lower when the fare is getting lower too. It's not proportional prefectly. But the lowest and the highest fare show us the opposite survival rate.

If you see a bar graph, you can see that almost all passengers paid under 100$.

---

위 표는 승객의 요금을 10$ 단위로 묶어서 보여주고 있습니다. 승객이 낸 요금이 적을 수록 생존 확률도 같이 줄어드네요. 완전히 비례하지는 않지만 최저 요금과 최고 요금은 반대의 생존률을 보여주고 있습니다.

그리고 막대 그래프를 보면 대부분 승객들은 100$ 이하의 금액을 내고 탑승한 것 같네요.

## Cabin(객실)


`Cabin` is having the most missing data in all of the fields. Before deciding how to fill those missing, let's see what normal data looks like.

---

`Cabin` (객실)은 모든 필드 중 가장 유실된 데이터가 많습니다. 어떻게 이 값들을 채울지 정하기 전에 정상적인 데이터가 어떻게 생겼는지 확인해 봅시다.

In [ ]:
df_train.groupby('Cabin').groups.keys()

There are lots of Cabins in the data. It's almost 150. And there is a capital alphabet in the front of the cabin name. It seems they are gathered in the same area or floors.

---

데이터에 많은 객실들이 있습니다. 거의 150개네요. 그리고 객실 이름 앞에 대문자 알파벳이 있습니다. 객실들이 같은 구역이나 층으로 모여있다는 것을 보여주는 것 같습니다.

In [ ]:
plt.figure(figsize=(8,30))
plt.title('Survival rate for each Cabin')
sns.countplot(y='Cabin', hue='Survived', data=df_train.sort_values('Cabin'), orient='h')

Graph above is showing us how many people were survived or not for each cabin

I think it's meanless that calculate the survival probability for each alphabets or cabins. Because each cabin has not more than 3 or 4 people. It's too small. And any particular cabins that start from the same alphabet didn't have a higher survival rate than other cabins.

But, we have to notice that if there are 2 or more passengers in the same cabin, they were survived with over 75% probability. And if there is 1 passenger in the cabin, they were survived with over 50% probability. passengers that can't be identified which cabin they stayed were survived only 30% of them

So, I will divide `Cabin` into 3 groups `group`, `solo`, and `unidentified`. Graph below shows us the survival rate each group.

---

위 그래프는 각 객실에서 얼마나 많은 사람들이 살아남고 죽었는지 보여줍니다.

저는 각 알파벳이나 객실별로 생존확률을 구하는 것은 무의미하다고 생각합니다. 왜냐하면 한 객실에는 많아봐야 3명에서 4명이 있었습니다. 수가 너무 적지요. 그리고 같은 알파벳으로 시작하는 선실들이 다른 선실들보다 높은 생존률을 가지고 있는 것도 아닙니다.

하지만, 우리는 2명 혹은 그 이상의 승객이 같은 객실에 있으면 75% 이상의 확률로 살아남았다는 것을 주목해야됩니다. 그리고 1명의 승객이 객실에 있으면 50% 이상의 확률로 살아남았습니다. 어떤 객실에 투숙했는지 알 수 없는 승객들은 겨우 30%만이 살아남았습니다.

그래서 저는 3개의 그룹, `group`, `solo`, 그리고 `unidentified` 으로 나눌겁니다. 아래 그래프는 각 그룹의 생존률을 보여줍니다.

In [ ]:
def groupCabin (row) :
    if (pd.isna(row.Cabin)) :
        return 0
    if (pd.concat([df_train['Cabin'], df_test['Cabin']]).value_counts()[row.Cabin] > 1) :
        return 2
    return 1

df_train['Cabin'] = df_train.apply(groupCabin, axis=1)
df_train['Cabin'] = df_train['Cabin'] \
    .astype('category') \
    .cat \
    .rename_categories(['unidentified', 'solo', 'group'])

g = df_train.groupby('Cabin').mean()['Survived'].plot.bar()
g.set_xticklabels(g.get_xticklabels(), rotation=0)
plt.ylim(top=1)
plt.show()

## Embakred (탑승항구)

There are three port that passengers embarked in. S(Southampton), C(Cherbourg), and Q(Queenstown).

`Embarked` has only two empties. Just fill empty values to 'S' because it's most common value

---

승객들이 탑승한 3개의 항구가 있습니다. S(Southampton), C(Cherbourg), and Q(Queenstown).

`Embarked` 에는 2개의 빈 값 밖에 없습니다. 그냥 빈 데이터들은 'S' 로 채우겠습니다. 왜냐하면 가장 흔한 값이거든요.

In [ ]:
print(df_train['Embarked'].value_counts())
print('Empties before filling : ', df_train['Embarked'].isna().sum())
df_train['Embarked'].fillna('S', inplace=True)
print('Empties after filling : ', df_train['Embarked'].isna().sum())
df_train['Embarked'] = df_train['Embarked'].astype('category')

# More Insight (더 많은 관점으로)

In [ ]:
columns = ['Sex', 'Pclass', 'SibSp', 'Parch', 'Cabin', 'Embarked', 'Fare', 'AgeGroupByTitle']

fig, axes = plt.subplots(3, 3, constrained_layout=True, figsize=(20,10))
for i, col in enumerate(columns) :
    if (i == 8):
        break
    sns.countplot(x=df_train[col], hue=df_train["Survived"], ax=axes[int(i / 3), int(i % 3)])
    axes[int(i / 3), int(i % 3)].legend(loc='upper right')

Graphs above are showing the survival probability for each feature. Blue bar shows us how many people were dead.

`Sex` looks like the most important feature. If you were male, it's not much chance you can survived. For other features, we can see that who had much more chance to survive.

With a simple calculation, you had the highest probability of survival if you were female, on first-class, had 1 sibling or married, had 1 parent or 1 child, stayed in cabin as group, paid highest fare, underage, and on aboard in Cherbourg.

But the reality is not that simple, let check how each feature affect otheres.

---

위 그래프는 각 특징에 대한 생존확률을 보여줍니다. 파란 막대는 얼마나 많은 사람들이 죽었는지 보여줍니다

`Sex` 가 가장 중요한 특징처럼 보입니다. 당신이 만약 남자였다면, 살 확률은 그지 높지 않았을 겁니다. 다른 특징들을 보면, 우리는 누가 더 살 확률이 높았는지 볼 수 있습니다.

간단한 계산으로, 당신이 만약 여자이고, 1등석이였으며, 1명의 형재자매가 있거나 결혼을 했고, 1명의 부모님 혹은 1명의 자식이 있으며, 단체로 객실에 있었고, 가장 높은 요금을 냈으며, 미성년자이고,  Cherbourg에서 탑승했다면 가장 살 확률이 높았습니다.

하지만 현실은 그리 간단하지 않습니다. 각 특징들이 다른 특징들에게 어떤 영향을 끼치는지 알아봅시다.

## Survival rate for each Sex and Age (나이와 성별에 따른 생존률)

In [ ]:
df_train.groupby(['AgeGroupByTitle', 'Sex', 'Survived'])['PassengerId'].count().unstack('Survived') \
    .plot(kind='bar', rot=0, figsize=(20, 4))

Before, we have checked the `Age` field and found that from 16 and older to under 55 years old have the highest death rate. and when we saw the survival rate for each sex, "male" shows us much higher death rate. In a way, it can be a natural result. Actually, "female" shows us higher survival rate than death rate for every age group. In "male" case, we can see that death rate is much higher except "kid".

---

우리는 이전에 `Age` 필드를 보면서 16세 이상부터 55세 미만까지가 가장 높은 사망률을 보여주는 것을 확인했습니다. 그리고 성별에 따른 사망률을 봤을 때는 남성이 훨씬 높은 사망률을 보여주었죠. 어찌보면 당연한 결과일 수 있습니다. 실제로 여성은 어떤 나이대에서든 사망률보다 생존률이 높다는 것을 보여주네요. 남성의 경우 아이를 제외하고는 사망률이 훨씬 높다는 것을 알 수 있습니다.

## 'Pclass', 'Fare', 'Embarked', and survival rate according to those fields.

## 'Pclass' (좌석등급), 'Fare' (요금), 'Embarked' (탑승항구), 그리고 해당 필드들에 따른 생존률

In [ ]:
df_train.groupby(['Fare', 'Pclass'])['Fare'].count().unstack('Pclass') \
    .plot(kind='bar', rot=0, stacked=True, figsize=(20, 4)) \
    .set_xlabel("Fare (10$)")

First, Graph above is `Pclass` according to `Fare`. Unit of X axis is 10\\$. Almost all passengers that paid under 10\\$ are third classes but there are first or second classes too. But over 80\\$, there are only first classes.

---

위 그래프는 승객이 낸 요금에 따른 좌석등급입니다. X축 단위는 10\\$ 입니다. 10\\$ 미만을 낸 승객들은 대부분 3등급이지만 1, 2등급도 없지는 않네요. 하지만 80\\$ 이상부터는 1등급 외에는 존재하지 않습니다.

In [ ]:
plt.figure(figsize=(20, 5))
plt.title('Embarked port for each class')
sns.countplot(x='Pclass', hue='Embarked', data=df_train)
plt.figure(figsize=(20, 5))
plt.title('Embarked port for each fare')
sns.countplot(x='Fare', hue='Embarked', data=df_train)

Graphs above are "Embarked port for each class" and "Embarked port for each fare". 

For reference, C = Cherbourg, Q = Queenstown, S = Southampton. And survival rates are C: 55%, Q: 39%, S: 34%.

First, Q is the most noticeable one. The higher the fare and pclass, the lower the number of passengers, the inverse proportion. It seems that it was not a rich town so much.

And then, S. In any pclass and fare, the most many passengers were aboard at this port. The town has the best transportation system I guess. Or it could varius people were distributed in this town from lower class to upper class. It looks the first one is a more probable.

Last, C. In this port, the first class passengers were the most. But the second class wasn't that much rather the number of third class passengers is similar to the number of first class passengers. Passengers that paid the highest fare were aboard at this port too. I don't know the details but it seems there was a gap between the rich and poor.

Now, let's figure out which port that the most passengers were survived from.

---

위 그래프들은 "Embarked port for each class" (각 등급 별 탑승항구) 와 "Embarked port for each fare" (각 요금에 따른 탑승항구) 입니다.

참고로, C = Cherbourg, Q = Queenstown, S = Southampton 입니다. 그리고 생존률은 C: 55%, Q: 39%, S: 34% 입니다.

일단 제일 눈에 띄는 것은 Q입니다. 요금과 좌성등급이 높아질 수록 승객은 줄어드는, 반비례의 형태를 띄고 있네요. 그렇게 부유한 동내는 아니였나 봅니다.

그 다음은 S 입니다. 어떤 좌석등급 혹은 요금이든 가장 많은 승객이 탑승한 항구입니다. 아마 제일 좋은 교통편을 가진 동내였나봅니다. 혹은 하류층부터 상류층까지 다양한 사람들이 분포하고 있었을 수도 있구요. 전자가 더 가능성 있어 보입니다.

마지막으로 C 입니다. 해당 항구에서는 1등급 승객이 제일 많았습니다. 하지만 2등급은 별로 보이지 않고 3등급의 승객들이 오히려 1등급 승객과 비슷한 숫자로 탔네요. 제일 높은 요금을 내고탄 승객들도 해당 항구에서 탑승했습니다. 자세히는 모르지만 빈부격차가 있었던 것 같습니다.

이제 어떤 항구에서 온 승객들이 많이 살아남았는지 봅시다.

In [ ]:
fig, axes = plt.subplots(1,4, figsize=(30, 5))
big_ax = fig.add_subplot(1,2,1)
fig.delaxes(axes[0])
fig.delaxes(axes[1])
big_ax.set_title("The first class's survival rate for each fare")
sns.countplot(x='Fare', hue='Survived', data=df_train[df_train['Pclass'] == 1], ax=big_ax)
axes[2].set_title("The second class's survival rate for each fare")
sns.countplot(x='Fare', hue='Survived', data=df_train[df_train['Pclass'] == 2], ax=axes[2])
axes[3].set_title("The third class's survival rate for each fare")
sns.countplot(x='Fare', hue='Survived', data=df_train[df_train['Pclass'] == 3], ax=axes[3])

Graphs above are each class's survival rate for each fare. Unit of x-axis is 10$ as before. Just know that y-axis maximum is different for each graph.

In almost case of the first class, survival rate is higher. But, from 0$ to 29$, the death rate is higher. This result make me doubt that they were the same first classes. Definitely, they were the first class. I wonder what happened to them at that time.

In case of second class, it's pretty similar. The death rate from under 19$, the survival from rate over 20$ is higher.

The third class is tragedy. Almost passengers were dead. They were paid a similar fare as second class but it shows us the opposite result.

Judging from that fact, if you had higher class and then paid higher fare, survival probability will be getting higher. I expected it to some extent, but I'm little sad because it seems like divide people's lives with money and class.

---

위 표들은 좌석등급들의 요금별 생존률입니다. 이전과 같이 x축은 10\\$ 단위입니다. 각 그래프의 y축은 최대치가 다르다는 것을 알아두세요.

1등급의 경우 대부분 생존률이 더 높습니다. 하지만 0\\$ 부터 29\\$ 까지는 사망률이 더 높네요. 같은 1등급이 맞는지 의심스럽습니다. 분명 같은 1등급 티켓인데, 당시에 무슨 일이 났었는지 궁금하네요.

2등급의 경우도 비슷합니다. 19\\$ 이하로는 사망률이 20\\$ 이상으로는 생존률이 더 높습니다.

3등급은 비극입니다. 대부분 사망했네요. 2등급 승객들과 비슷한 요금을 냈지만 전혀 다른 결과를 보여주네요.

위 사실들을 미루어 볼 때 일단 등급이 높아야되고 그 다음은 요금을 많이 낼 수록 살 확률이 높습니다. 어느정도 예상은 했지만 돈과 등급으로 사람의 목숨을 나누는 것 같아서 약간 슬퍼지네요.

## The similarity between SibSp and Parch (SibSp 와 Parch 값의 유사성)

`SibSp` is number of siblings and spouse aboard the titanic and `Parch` is number of parents or children aboard the titanic. Definetly, both of fields have difference but there are same under the name of family. let's figure out correlation of both fields

---

`SibSp`는 타이타닉에 탑승한 형재자매 그리고 배우자의 수, `Parch`는 타이타닉에 탑승한 부모 혹은 자식의 수를 나타내는 값입니다. 분명 차이점은 있지만 가족이라는 이름으로는 모두 하나입니다. 이 둘의 값의 상관관계를 알아봅시다.

In [ ]:
df2 = df_train[(df_train['SibSp'] > 0) | (df_train['Parch'] > 0)]
sns.heatmap(df2[['SibSp', 'Parch']].corr(), annot=True, square=True)

Graph above shows us each column's similarity. They are matched only 11%. That means about 10% of the passengers came with their siblings, spouses, parents and, children. Not a few passengers came with their families. It's too bad to finish here, so I'll find those 10% passenger's survival rate.

---

위 표는 서로의 유사도를 나태내고 있습니다. 둘은 11% 정도만 일치하네요. 그말인 즉슨 형재자매와 배우자 그리고 부모님 혹은 자식 모두와 온 승객은 10% 정도입니다. 적지 않는 승객들이 가족들을 대리고 왔네요. 여기서 끝내기는 아쉬우니 저 10%의 생존률을 구해보겠습니다.

In [ ]:
df2 = df_train[(df_train['SibSp'] > 0) & (df_train['Parch'] > 0)]
df2['family'] = (df2['SibSp'] + df2['Parch'])
sns.countplot(x='family', hue='Survived', data=df2)

It's so sad. Fortunately, up to three people have a much higher survival rate. But death rate is much higher for more than 4 people, all of them died for more than 7.

---

정말 안타깝습니다. 다행히 3명까지는 생존률이 훨씬 높습니다. 하지만 4명 이상부터는 사망률이 훨씬 높고 7명 이상부터는 모두 죽었네요.

# Build Model & Prediction (모델 구축 & 예측)

So far, by analyzing the data, we've looked at how or how each feature affects survival rates or other features.

Now let's build a model based on this analysis and predict whether a passenger lives or dies with the given value.

---

지금까지 데이터들을 분석함으로써 각 특징들이 생존률 혹은 다른 특징들에게 얼마나 혹은 어떻게 영향을 끼치는지 알아봤습니다.

이제는 이 분석을 기반으로 모델을 구축하고 주어진 값으로 승객이 살았는지 죽었는지 예측해봅시다.

In [ ]:
df2 = df_train

X = df2[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'AgeGroupByTitle']]
X['Sex'] = X['Sex'].cat.codes
X['Cabin'] = X['Cabin'].cat.codes
X['Embarked'] = X['Embarked'].cat.codes
X['AgeGroupByTitle'] = X['AgeGroupByTitle'].cat.codes
X.head(5)

We've been through a lot of fields so far. Some of the fields that are considered meaningful for prediction are as follows :

`Pclass`, `SexSibSp`, `Parch`, `Fare`, `Cabin`, `Embarked`, `AgeGroupByTitle`

First, change the categorized text forms fields into numerical forms so that model can calculate them.

---

우리는 지금까지 많은 필드들을 거쳐갔는데요. 그 중 예측에 의미 있다고 판단되는 필드들은 다음과 같습니다.

`Pclass`, `SexSibSp`, `Parch`, `Fare`, `Cabin`, `Embarked`, `AgeGroupByTitle`

일단 텍스트 형태로 카테고리화 되어 있는 필드들을 계산을 할 수 있도록 숫자 형태로 바꿔줍니다.

In [ ]:
X = X.to_numpy()
y = df2[['Survived']].to_numpy()
X[0:5], y[0:5]

`X` is the training data, `y` is result of the training data.

Both fields convert to 'ndarray' objects in the 'numpy' library.

After converting it to 'ndarray', the printed result is the same as above

---

`X`는 훈련 데이터, `y`는 해당 훈련 데이터의 결과값입니다.

두 필드 모두 `numpy` 라이브러리의 `ndarray` 오브젝트로 변환해줍니다.

`ndarray` 형태로 변환한 후 출력해보면 위와 같습니다.

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

Define a sigmoid function to use as activation function

---

활성화 함수로 사용할 sidmoid function 을 선언해줍니다.

In [ ]:
def hypothesis (X) :
    a1 = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
    z2 = np.dot(a1, weights1.T)
    a2 = sigmoid(np.concatenate((np.ones((z2.shape[0], 1)), z2), axis=1))
    z3 = np.dot(a2, weights2.T)
    h = sigmoid(z3)
    return h

The model will use neural networks for prediction. The above is a neural network as a code. It consists of one input layer, one hidden layer, and one output layer.

---

예측에는 신경망을 사용할 것입니다. 위는 신경망을 코드로 옯겨놓은 것이며 한 개의 입력층, 한 개의 은닉층, 그리고 하나의 출력층으로 이루어져 있습니다.

In [ ]:
def get_cost (weights1, weights2) :
    m, n = X.shape
    
    # get hypothesis's result
    a1 = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
    z2 = np.dot(a1, weights1.T)
    a2 = sigmoid(np.concatenate((np.ones((z2.shape[0], 1)), z2), axis=1))
    z3 = np.dot(a2, weights2.T)
    h = sigmoid(z3)
    
    # calculate cost
    cost = np.sum(y * np.log(h) + (1-y) * np.log(1-h)) / (-m) + np.sum(np.concatenate((weights1[:, 1:].flatten() ** 2, weights2[:, 1:].flatten() ** 2))) * lamb / (2 * m)
    
    # get error value for each layer
    d3 = h - y
    d2 = np.dot(d3, weights2) * a2 * (1-a2)
    
    # calcaulate partial derivative
    D1 = np.dot(d2[:, 1:].T, a1)
    D2 = np.dot(d3.T, a2)
    grad1 = D1 / m + lamb / m * np.concatenate((np.zeros((weights1.shape[0], 1)), weights1[:, 1:]), axis=1)
    grad2 = D2 / m + lamb / m * np.concatenate((np.zeros((weights2.shape[0], 1)), weights2[:, 1:]), axis=1)

    return cost, grad1, grad2

The above function returns the value of the hypothesis error value and the partial derivative of weights. The details are described in the comments of the code.

---

위 함수는 가설의 오류치와 가중치의 편미분된 값을 돌려줍니다. 자세한 내용은 코드의 주석으로 설명되어 있습니다.

In [ ]:
# set lambda vaule for regularization
lamb = 0.5

# random initialization
init_epsilon = 0.1
weights1 = np.random.rand(8, 9) * 2 * init_epsilon - init_epsilon
weights2 = np.random.rand(1, 9) * 2 * init_epsilon - init_epsilon

# gradient descent for 1,000 times iteration
cost_history = pd.Series()
for i in range(0, 1000) :
    cost, grad_weights1, grad_weights2 = get_cost(weights1, weights2)
    weights1 = weights1 - grad_weights1
    weights2 = weights2 - grad_weights2
    cost_history.at[i] = cost

# visualize cost history for checking it's working correctly
cost_history.plot()
print(cost_history.iat[-1])

Finally, start learning using gradient descent.

The study was 1,000 times, and the above table shows the error after each study. It seems to have an error value of about 0.45 after learning. The closer you get to zero, the more there is no error. The hypothesis doesn't look perfect. I will continue to improve it.

---

마지막으로 graident descent를 이용하여 학습을 시작합니다.
 
학습은 1,000번을 진행하였으며 위 표는 각 학습 이후의 오류치를 표로 나태난 것입니다. 학습 이후에는 약 0.45 정도의 오류치를 가지고 있는 것으로 보입니다. 0으로 가까워질수록 오류가 없는 것입니다. 가설이 완벽해보이지는 않네요. 앞으로도 계속 개선해봐야겠습니다.

In [ ]:
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

df_test['Sex'] = df_test['Sex'].astype('category')

df_test['AgeGroupByTitle'] = df_test.apply(groupAgeByTitle, axis=1)
df_test['AgeGroupByTitle'] = df_test['AgeGroupByTitle'] \
    .astype('category') \
    .cat \
    .rename_categories(['unidentified', 'kid', 'adult', 'elder'])

df_test['Cabin'] = df_test.apply(groupCabin, axis=1)
df_test['Cabin'] = df_test['Cabin'] \
    .astype('category') \
    .cat \
    .rename_categories(['unidentified', 'group', 'solo'])

df_test['Fare'] = df_test.apply(groupCabin, axis=1)
df_test['Fare'] = (df_test['Fare'] / 10).astype(int)

df_test['Embarked'].fillna('S', inplace=True)
df_test['Embarked'] = df_test['Embarked'].astype('category')

X = df_test[['PassengerId', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'AgeGroupByTitle']]
X['Sex'] = X['Sex'].cat.codes
X['Cabin'] = X['Cabin'].cat.codes
X['Embarked'] = X['Embarked'].cat.codes
X['AgeGroupByTitle'] = X['AgeGroupByTitle'].cat.codes

X = X.to_numpy()

result = pd.DataFrame([], index=['PassengerId', 'Survived'])
for x in X :
    PassengerId = x[0]
    x = x[1:]
    r = hypothesis(np.array([x]))
    if (r[0,0] >= 0.5) :
        r = 1
    else :
        r = 0
    result = result.append({ 'PassengerId': PassengerId, 'Survived': r }, ignore_index=True)
result = result[2:] # fix it after
result['PassengerId'] = result['PassengerId'].astype('int')
result['Survived'] = result['Survived'].astype('int')
result.to_csv('./result.csv', index=False)